# 1. 机器学习中的优化思想

在之前的学习中，我们建立神经网络时总是先设定号w与b的值（或者由我们调用的PyTorch类帮助我们随机生成权重向量w），接着通过加和求出z，再在z上嵌套sigmoid或者softmax函数，最终获得神经网络的输出。我们的代码及计算流程，总是从神经网络的左侧向右侧计算的。之前我们提到过，这是神经网络的正向传播过程。但很明显，这并不是神经网络算法的全流程，这个流程虽然可以输出预测结果，但却无法保证神经网络的输出结果与真实值接近。

![Alt text](image-9.png)

在讲解线性回归时，我们提起过，线性回归的任务就是构造一个预测函数来映射输入的特征矩阵X和标签值y的线性关系。构造预测函数核心就是找出模型的权重向量W，并令线性回归的输出结果与真实值接近，也就是求解线性方程组中的w和b。对神经网络而言也是如此，我们的核心任务是求解一组最适合的w和b，令神经网络的输出结果与真实值接近。找寻这个w和b的过程就是"学习“,也叫做”训练“或者”建模“。

    模型训练的目标
    求解一组最合适的权重向量W，令神经网络的输出结果与真实值尽量接近

那我们如何评价w和b是否合适呢？我们又如何衡量我们的输出结果与真实值之间的差异大小呢？此时，我们就需要使用机器学习中通用的优化流程了。在讲解autograd的时候，其实我们就已经提过这个优化流程了。

![Alt text](image-10.png)

1. 提出基本模型，明确目标

我们的基本模型就是我们自建的神经网络架构，我们需要求解的就是神经网络架构中的权重向量W

2. 确定损失函数、目标函数

我们需要定义某个评估指标，用以衡量模型权重为W的情况下，预测结果与真实结果的差异。当真实值与预测值差异越大时，我们就认为神经网络学习过程中丢失了许多信息，丢失的这部分被形象地称为”损失“,因此评估真实值与预测值差异的函数被我们称为”损失函数“

    * 关键概念：损失函数*
    衡量真实值与预测结果的差异，评价模型学习过程中产生的损失函数
    在数学上，表示为以需要求解的权重向量W为自变量的函数L（w）
    如果损失函数的值很小，则说明模型预测值与真实值很接近，模型在数据集上表现优秀，权重优秀
    如果损失函数值大，则说明模型预测值与真实值差异很大，模型在数据集上表现差劲，权重糟糕

我们希望损失函数越小越好，以此，我们将问题转变为求解函数L（w）的最小值所对应的自变量W。但是，损失函数往往不是一个简单的函数，求解复杂函数就需要复杂的数学工具。在这里，我们使用的数学工具可能有两部分
 
* 将损失函数L（w）转变成凸函数的数学方法，常见的有拉格朗日变换等
* 在凸函数上求解L（w）的最小值对应的w的方法，也就是以梯度下降为代表的优化算法

3. 确定合适的优化算法


4. 利用优化算法，最小化损失函数，求解最佳权重W（训练）

之前我们在线性回归上走过这个流程。对于线性回归，我们的损失函数是SSE

In [1]:
import torch
from torch.nn import MSELoss    # class

yhat = torch.randn(size=(50,), dtype= torch.float32)
y = torch.randn(size=(50,), dtype= torch.float32)


In [3]:
criterion = MSELoss()
loss = criterion(yhat, y)
loss

tensor(1.8920)

In [ ]:
# 关于MSELoss类，只需要注意一个参数 reduction
MSELoss(reduction= "mean/ sum") # mean参数：MSE    sum参数：SSE

# 2. 二分类交叉熵损失函数

这一节，我们将介绍二分类神经网络的损失函数：二分类交叉熵损失函数（Binary Cross Entropy Loss），也叫对数损失。这个损失函数被广泛地使用再任何输出结果是二分类的神经网络中，即不止限于单层神经网络，还可以被拓展到多分类中，因此理解二分类交叉熵损失是非常重要的一环。大多数时候，除非特殊声明为二分类，否则提到交叉熵损失，我们就会默认算法的分类目标是多分类。

二分类交叉熵损失函数是由极大似然估计推导出来的，对于有m个样本的数据集而言，在全部样本上的整体损失写作：

$$
L(w)=-\sum_{i=1}^m\left(y_i * \ln \left(\sigma_i\right)+\left(1-y_i\right) * \ln \left(1-\sigma_i\right)\right)
$$

在单个样本的损失写作：
$$
L(w)_i=-\left(y_i * \ln \left(\sigma_i\right)+\left(1-y_i\right) * \ln \left(1-\sigma_i\right)\right)
$$

**极大似然估计的感性认识：**
如果一个事件的发生概率很大，那么这个事件应该很容易发生。如果我们希望一件事发生，就应该增加这个事件发生的概率。假设现在存在事件A，其发生概率受权重W的影响，那为了事件A尽可能地发生，那我们只要寻找令其发生概率最大化的权重W。寻找相应的权重W，使得事件发生的概率最大，就是极大似然估计的基本方法。

* 推导

二分类神经网络的标签是[0, 1]，次标签服从伯努利分布（即0-1分布），因此可得：

样本 i 在由特征向量 Xi 和权重向量 W 组成的预测函数中，样本标签被预测为1的概率为：
$$
P_1=P\left(\hat{y}_i=1 \mid \boldsymbol{x}_i, \boldsymbol{w}\right)=\sigma
$$
对二分类而言，$ \sigma $ 就是sigmoid函数的结果

样本 i 在由特征向量 Xi 和权重向量 W 组成的预测函数中，样本标签被预测为 0 的概率为：

$$
P_0=P\left(\hat{y}_i=0 \mid \boldsymbol{x}_{\boldsymbol{i}}, \boldsymbol{w}\right)=1-\sigma
$$

将两种取值的概率整合，我们可以定义如下等式：

$$
P\left(\hat{y}_i \mid \boldsymbol{x}_i, \boldsymbol{w}\right)=P_1^{y_i} * P_0^{1-y_i}
$$

这个等式同时代表了P1,P0，在数学上，它被叫做逻辑回归的假设函数

当样本 i 的真实标签 yi 为 1 的时候，1-yi 就等于0，P0的0次方就是1，所以 $ P\left(\hat{y}_i \mid \boldsymbol{x}_i, \boldsymbol{w}\right) $ 就等于 P1，这个时候，如果 P1为1，模型的效果就很好，损失就很小。

同理，当 yi 为0的时候，$ P\left(\hat{y}_i \mid \boldsymbol{x}_i, \boldsymbol{w}\right) $ 就等于 P0，此时如果 P0 非常接近1，模型的效果就很好，损失就很小

所以，为了达成让模型拟合好，损失小的目的，我们每时每刻都希望 $ P\left(\hat{y}_i \mid \boldsymbol{x}_i, \boldsymbol{w}\right) $ 的值等于1。而 $ P\left(\hat{y}_i \mid \boldsymbol{x}_i, \boldsymbol{w}\right) $ 的本质是样本 i 由特征向量 xi和权重w组成的预测函数中，预测出所有可能的 $ \hat{y}_i $ 的概率，因此 1 就是它的最大值。也就是说，每时每刻，我们都在追求 $ P\left(\hat{y}_i \mid \boldsymbol{x}_i, \boldsymbol{w}\right) $ 的最大值。而寻找相应的参数 W ，使得每次得到的预测概率最大，正是极大似然估计的基本方法，不过 $ P\left(\hat{y}_i \mid \boldsymbol{x}_i, \boldsymbol{w}\right) $ 是对单个样本而言的，因此我们还需要将其拓展到多个样本上

$ P\left(\hat{y}_i \mid \boldsymbol{x}_i, \boldsymbol{w}\right) $ 是对单个样本 i 而言的函数，对一个训练集的 m 个样本来说，我们可以定义如下等式来表达所有样本在特征张量 X和权重向量 W 组成的预测函数中，预测出所有可能的 $ \hat{y}_i $ 的概率 p 为：

$$
\begin{aligned}
\boldsymbol{P} & =\prod_{i=1}^m P\left(\hat{y}_i \mid x_i, w\right) \\
& =\prod_{i=1}^m\left(P_1^{y_i} * P_0^{1-y_i}\right) \\
& =\prod_{i=1}^m\left(\sigma_i^{y_i} *\left(1-\sigma_i\right)^{1-y_i}\right)
\end{aligned}
$$

这个函数就是逻辑回归的似然函数。对该概率 P 取以e为底的对数，再由 $\log (A * B)=\log A+\log B$ 和 $\log A^B=B \log A$ 可得到逻辑回归的对数似然函数：

$$
\begin{aligned}
\ln \boldsymbol{P} & =\ln \prod_{i=1}^m\left(\sigma_i^{y_i} *\left(1-\sigma_i\right)^{1-y_i}\right) \\
& =\sum_{i=1}^m \ln \left(\sigma_i^{y_i} *\left(1-\sigma_i\right)^{1-y_i}\right) \\
& =\sum_{i=1}^m\left(\ln \sigma_i^{y_i}+\ln \left(1-\sigma_i\right)^{1-y_i}\right) \\
& =\sum_{i=1}^m\left(y_i * \ln \left(\sigma_i\right)+\left(1-y_i\right) * \ln \left(1-\sigma_i\right)\right)
\end{aligned}
$$

这就是我们的二分类交叉熵函数。为了数学上的便利以及更好地定义“损失”的含义，我们希望将极大值问题转换为极小值问题，因此我们对 lnP 取负，并且让权重 w 作为函数的自变量，就得到了我们的损失函数 L（w）：

$$
L(w)=-\sum_{i=1}^m\left(y_i * \ln \left(\sigma_i\right)+\left(1-y_i\right) * \ln \left(1-\sigma_i\right)\right)
$$

现在，我们已经将模型拟合中的“最小化损失”问题，转换成了对函数求解极值的问题。

In [4]:
import torch
# loss = -(y*ln(sigma) + (1-y) * ln(1 - sigma))
# z = XW
# X, W
# m：样本量

m = 3 * pow(10, 3)
torch.random.manual_seed(420)
X = torch.rand((m, 4), dtype = torch.float32)
w = torch.rand((4, 1), dtype = torch.float32)
y = torch.randint(low=0, high= 2, size= (m, 1), dtype = torch.float32)

zhat = torch.mm(X, w)
sigma = torch.sigmoid(zhat)
sigma.shape

torch.Size([3000, 1])

In [5]:
loss = -(y*torch.log(sigma) + (1-y)*torch.log(1 - sigma))
loss

tensor([[0.3075],
        [0.3073],
        [0.9198],
        ...,
        [0.3876],
        [0.4536],
        [0.3442]])

In [2]:
import time

In [6]:
start = time.time()
sum(loss)
now = time.time()
print(now - start)

0.010477781295776367


In [11]:
start = time.time()
torch.sum(loss)/m       # 使用torch中的函数计算，会比普通的函数计算快很多很多
now = time.time()
print(now - start)

0.0


# 3. 使用PyTorch来实现交叉熵损失

## 3.1 方法1：nn模块中的类

        class BCEWithLogitsLoss
        class BCELoss

对于二分类交叉熵损失，nn提供了两个类：BCEWithLogitsLoss 以及 BCELoss。虽然 PyTorch 官方没有直接明确，但实际上两个函数所需要输入的参数不同。

BCEWithLogitsLoss 内置了 sigmoid 函数与交叉熵函数，它会自动计算输入值的 sigmoid值，因此需要输入zhat 与真实标签，且顺序不能变化，zhat必须在前。

相对的，BCELoss 中只有交叉熵函数，没有sigmoid层，因此需要输入sigma与真实标签，且顺序不能变化。

同时，这两个函数都要求预测值与真实标签的数据类型及结构（shape）必须相同，否则运行就会报错。

In [12]:
import torch 
import torch.nn as nn

criterion = nn.BCELoss()
loss = criterion(sigma, y)
loss

tensor(0.7962)

In [14]:
criterion_2 = nn.BCEWithLogitsLoss()
loss = criterion_2(zhat, y)
loss


tensor(0.7962)

BCEWithLogitsloss 在精度上比 sigmoid函数的精度更高

## 3.2 functional库中的计算函数

    function F.binary_cross_entropy_with_logits
    function F.binary_cross_entropy

和 nn 中的类相似，名称中带有 Logits的是内置了 sigmoid功能的函数，没有带 Logits的，是只包含了交叉熵损失的函数。对于含有 sigmoid 功能的函数，我们需要的输入是zhat与标签，不含sigmoid的函数我们则需要输入sigma与标签。